## ファイルとディレクトリ
1. GoogleDriveの内容をGoogleColaboratoryから読み書きできるようにGoogleDriveをマウントします。<br>
2. アップロードしたHTMLファイルをPythonで読みとります。<br>
3. GoogleSpreadSheetをGoogleDrive上に作成します。<br>
4. 読み取ったデータを作成したGoogleSpreadSheetに書き込みます。

## GoogleDriveをGoogleColaboratoryから開く
GoogleColaboratoryのディレクトリにファイルを書き込んでも、<br>
**30分でファイルが消えてしまう。**<br>
GoogleColaboratoryで永続的なGoogleDriveをマウントする。<br>
マウント（乗り上げる）することで、GoogleColaboratoryからGoogleDriveが使えるようになる。

In [ ]:
from google.colab import drive
googledrive = '/content/drive'                                   # GoogleColaboratory の マウントポイント
drive.mount( googledrive )                                        # マウントポイントにGoogleDriveをマウントする
mydrive = googledrive + '/MyDrive'                           # 上でマウントしたGoogleDriveのMyDriveの絶対path
pylearn_dir = mydrive + "/Colab Notebooks/pylearn" # 空白がある場合は、ダブルクォーテーションで囲う

## GogoleDrive を自作ライブラリやデータの置き場として使う
 - 自作ライブラリをGoogleDriveの次のフォルダに格納
 ```
 'マイドライブ' > 'Colab Notebooks' > 'pylearn'
 ```

- sys ライブラリで Pythonが読み込むフォルダに、自作ライブラリ格納フォルダを追加する
```
import sys
sys.path.append(自作ライブラリ格納フォルダ)
```


In [ ]:
# ライブラリパスの場合は、最後にディレクトリ記号の '/' （スラッシュ）をつける
pylearn_path = mydrive + "/Colab Notebooks/pylearn/"# 空白がある場合は、ダブルクォーテーションで囲う

import sys
sys.path.append(pylearn_path)                                    # Pythonが参照するpathを追加
import evalrewardshrink as er                                     # 自作のPythonライブラリを読み込む

# いろいろな形式で金額が書かれた文字列のリスト
words = [
           "700,000円以上"
         , "月額～800,000円"
         ]

# evalreward.pyを使って金額を数値に変換する
for word in words:
    reward = er.get_reward(word)
    print('reward == ', reward)

f = open(pylearn_path + "evalrewardshrink.py", 'r', encoding='utf-8')
print(f.read())
f.close()

## ライブラリをインストールする
- Playwrightをインストール
playwrightのインストール先は、
```
/usr/local/lib/pyhon3.7/dist-packages/playwright
```

パッケージのインストール先
```
!pip show <package name>
```

```
Name: playwright
Version: 1.20.0
Summary: A high-level API to automate web browsers
Home-page: https://github.com/Microsoft/playwright-python
Author: Microsoft Corporation
Author-email: 
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: websockets, pyee, typing-extensions, greenlet
Required-by: 
```


In [ ]:
!pip install playwright
!playwright install-deps
!playwright install webkit

In [ ]:
!pip show playwright

## 新しいGoogleSpreadSheetを作成する
GoogleDrive内のエクセルファイルを読み出す<br>
GoogleDrive上にあるGoogleSpreadSheetは、権限付与にてアクセスする

In [ ]:
from google.colab import auth                                             # Google Colaboratory に 自分の GoogleDrive のアクセス権を移譲する
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
from apiclient import discovery

destFolderId = '1NZWQxKeM-DkWWEJSSvnNrYs5Z_wKtnFX'  # 新しいGoogleSpreadSheetを作成するフォルダの ID を指定
title = 'joboffer_list'                                                                # GoogleSpreadSheetの名前を指定
credentials = GoogleCredentials.get_application_default()       # クレデンシャル（資格情報）を取得

drive_service = discovery.build('drive', 'v3', credentials=credentials)  # クレデンシャルを使って新しいGoogleSpreadSheetの作成先を構成
file_metadata = {
    'name': title,
    'mimeType': 'application/vnd.google-apps.spreadsheet',
    'parents': [destFolderId]
}
file = drive_service.files().create(body=file_metadata).execute()  # 作成した GoogleSpreadSheet の情報が返ってくる
print(file) # 戻り値の辞書を印字する
print('kind:', file['kind'])
print('id:', file['id'])
print('name:', file['name'])
print('mimeType:', file['mimeType'])
# idを採っておく
# {'kind': 'drive#file', 'id': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX', 'name': 'joboffer_list', 'mimeType': 'application/vnd.google-apps.spreadsheet'}


### シートを作成
作成したGoogleSpreadSheetに新しいシートを追加し、初期にある 'シート1' を削除する

In [ ]:
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
id='1HSXd-ijIZqRCeGJw6vNIKVmFinvJtJsP2G7Lk-ceRLs' # 作成した GoogleSpreadSheet のid 
url='https://docs.google.com/spreadsheets/d/'+file['id']       # Google Spread Sheet のURL
workbook = gc.open_by_url(url)
workbook.add_worksheet(title='entry', rows=1000, cols=10)
workbook.del_worksheet(workbook.sheet1)

## アップロードしたHTML文書をスクレイピングしてGoogleSpreadSheetに書き込む

- スクレイピングライブラリ Playwright<br>
Node.js、Python、Java、.NET版がある<br>
[Playwright公式ドキュメントPython版](https://playwright.dev/python/docs/intro)

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
from playwright.async_api import async_playwright
from google.colab import auth      # Google Colaboratory に 自分の GoogleDrive のアクセス権を移譲する
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
import numpy as np
googledrive = '/content/drive'
mydrive = '/content/drive/MyDrive'
html_dir = mydrive + '/html'           # /content/drive/MyDrive/html

gc = gspread.authorize(GoogleCredentials.get_application_default())
id='1HSXd-ijIZqRCeGJw6vNIKVmFinvJtJsP2G7Lk-ceRLs' # 作成した GoogleSpreadSheet のid 
url='https://docs.google.com/spreadsheets/d/'+id               # Google Spread Sheet のURL
worksheet = gc.open_by_url(url).worksheet('entry')

# ベリフィケーションコードをテキスト入力域に貼り付ける
base_url = 'https://www.wantedly.com'
async def main():
    async with async_playwright() as p:
        browser = await p.webkit.launch()
        page = await browser.new_page()

        for i in range(82):

            await page.goto('file://'+ html_dir + '/' + str( i + 1 ).zfill(2) + '.html')
            locator = page.locator("body > div > article")
            count = await locator.count()

            for j in range(count):

                jd_locator = locator.locator("div > div.project-index-single-inner > div.project-detail > h1 > a")
                jd_element = jd_locator.nth( j )
                jd_anchor = await jd_element.inner_text()
                jd_href = await jd_element.get_attribute('href')

                co_locator = locator.locator("div > div.project-bottom > div.company-name > h3 > a")
                co_element = co_locator.nth( j )
                co_anchor = await co_element.inner_text()
                co_href = await co_element.get_attribute('href')
                print('href:', co_href)

                sheet_range = 'B' + str( ( i * 10 )  + j + 2 ) + ':E' + str( ( i * 10 ) + j + 2 )
                cell_list = worksheet.range( sheet_range )
                cell_list[0].value = co_anchor
                cell_list[1].value = base_url + co_href
                cell_list[2].value = jd_anchor
                cell_list[3].value = base_url + jd_href
                worksheet.update_cells(cell_list)

        await browser.close()

asyncio.run(main())

## GoogleSpreadSheetから読み出したURLをスクレイピングして結果を書き込む
- GoogleSpreadSheetからURLを読み出す
- URLをスクレイピングして情報を抽出する
- スクレイピングした情報をGoogleSpreadSheetに書き込む

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
import numpy as np
import asyncio
from playwright.async_api import async_playwright
from google.colab import drive
googledrive = '/content/drive'
drive.mount(googledrive)
id='1HSXd-ijIZqRCeGJw6vNIKVmFinvJtJsP2G7Lk-ceRLs' # 作成した GoogleSpreadSheet のid 

gc = gspread.authorize(GoogleCredentials.get_application_default())
url='https://docs.google.com/spreadsheets/d/' + id # Google Spread Sheet のURL
worksheet = gc.open_by_url(url).worksheet('entry')
# ベリフィケーションコードをテキスト入力域に貼り付ける

# get_all_values gives a list of rows.
values = worksheet.get_all_values()

import pandas as pd
pd.options.display.float_format = '{:}'.format

df = pd.DataFrame.from_records(data=values[1:])
df = df.replace({1: {'' : ''}}) # B列の値が空文字の場合は、0を設定
df = df.astype({
    0: str, 1: str, 2: str, 3: str, 4: str
})
# A=0, B=1, C=2, D=3, E=4, F=5, G=6, H=7, I=8, J=9, K=10, L=11, M=12 

async def main():
    async with async_playwright() as p:
        browser = await p.webkit.launch()
        page = await browser.new_page()
        page.route("**/*", lambda route: route.abort()
            if route.request.resource_type == "image"
            else route.continue_()
        )
        for i, item in df.iterrows():
            if item[4] != '':
                await page.goto(item[4])
                section_element = page.locator('#project-show-body > div > div.column-main > div > div > section:nth-child(1) > div')
                section = await section_element.inner_text()
                link_element = page.locator('#project-show-body > div > div.column-side > div > section.new-column-side-section.company-section > div > div.company-info-list > li:nth-child(1) > div.company-description > a')
                link = await link_element.get_attribute('href')

                worksheet.update_cell( i + 2, 6, section )
                worksheet.update_cell( i + 2, 7, link )

        await browser.close()

asyncio.run(main())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: coroutine 'Page.route' was never awaited
